# Import

In [16]:
import time

In [1]:
import pandas as pd

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

/home/container_user/.cache/pypoetry/virtualenvs/hansoldeco3-QckXwucX-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import transformers
import accelerate
print(transformers.__version__)
print(accelerate.__version__)

4.49.0
1.4.0


In [3]:
print("CUDA 사용 가능:", torch.cuda.is_available())
print("GPU 개수:", torch.cuda.device_count())
print("사용 중인 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU 사용 중")

CUDA 사용 가능: True
GPU 개수: 1
사용 중인 GPU: NVIDIA GeForce RTX 3080


# Data Load & Pre-processing

In [4]:
path_train = "/workspace/Storage/hansoldeco3/Data/train.csv"
path_test = "/workspace/Storage/hansoldeco3/Data/test.csv"

In [5]:
train = pd.read_csv(path_train, encoding = 'utf-8-sig')
test = pd.read_csv(path_test, encoding = 'utf-8-sig')

In [6]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [7]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [8]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

# Model import

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [10]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

print(model.hf_device_map)  # GPU 정상 할당 여부 확인

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.93s/it]


{'': 0}


# Vector store 생성

In [13]:
# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"  # 임베딩 모델 선택
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})


# RAG chain 생성

In [14]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,  # sampling 활성화
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
)

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

{context}

### 질문:
{question}

[/INST]

"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 단순 컨텍스트 결합 방식 사용
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}  # 커스텀 프롬프트 적용
)

Device set to use cuda:0
/tmp/ipykernel_37580/3752858109.py:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


# Inference

In [17]:
# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

for idx, row in combined_test_data.iterrows():

    t = time.time()
    print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")
    print(row['question'])
    # RAG 체인 호출 및 결과 생성
    prevention_result = qa_chain.invoke(
        row['question'],
        # llm_kwargs={"max_new_tokens": 50, "use_cache": True, "do_sample": False}
        )

    # 결과 저장
    result_text = prevention_result['result']
    test_results.append(result_text)
    print(f"iter {idx} 완료 : {time.time()-t}초")
print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964

[샘플 1/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설기계'(중분류: '콘크리트펌프')와 관련된 사고가 발생했습니다. 작업 프로세스는 '타설작업'이며, 사고 원인은 '펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 0 완료 : 34.222492933273315초

[샘플 2/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '수장공사' 작업에서 사고객체 '건설공구'(중분류: '공구류')와 관련된 사고가 발생했습니다. 작업 프로세스는 '절단작업'이며, 사고 원인은 '작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 1 완료 : 26.261958837509155초

[샘플 3/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '미장공사' 작업에서 사고객체 '기타'(중분류: '기타')와 관련된 사고가 발생했습니다. 작업 프로세스는 '이동'이며, 사고 원인은 '작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 2 완료 : 28.532641649246216초

[샘플 4/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '조적공사' 작업

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


iter 9 완료 : 30.535161018371582초

[샘플 11/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '조적공사' 작업에서 사고객체 '기타'(중분류: '기타')와 관련된 사고가 발생했습니다. 작업 프로세스는 '준비작업'이며, 사고 원인은 '고령의 작업자가 현장에 배치되면서 발을 삐끗해서 넘어지는 사고로 ,  현장 작업자의 나이제한, 건강상태 점검 등 수시로 확인이 필요한 상황임'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 10 완료 : 28.062204122543335초

[샘플 12/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '토목', 중분류 '지반개량공사' 작업에서 사고객체 '건설기계'(중분류: '항타 및 항발기')와 관련된 사고가 발생했습니다. 작업 프로세스는 '이동'이며, 사고 원인은 '장비이동을 위해 설치된 철판 궤도 이탈(감리자 의견)'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 11 완료 : 30.6518235206604초

[샘플 13/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '수장공사' 작업에서 사고객체 '건설자재'(중분류: '자재')와 관련된 사고가 발생했습니다. 작업 프로세스는 '운반작업'이며, 사고 원인은 '합판 자재 운반작업 중 겹쳐저 있던 자재를 빼기 위해 힘을 주어 자재를 빼려던 중 잡고있던 합판 자재 1장이 급히 신체 뒷편에 있던 거실벽체쪽으로 빠지면서 합판 자재를 잡고있던 오른손 중지가 순간적으로 거실 벽체와 합판사이에 눌리고 부딪힘.'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 12 완료 : 31.095176458358765초

[샘플 14/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '수장공사' 작업에서 사고객

# Submission

In [18]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

(964, 768)


In [20]:
submission = pd.read_csv('/workspace/Storage/hansoldeco3/Data/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')